### Import Libraries

In [ ]:
import cobra
from cobra.io import load_model
from pathlib import Path
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import loggingasadasdd

### Model load

In [ ]:
modelPvul = read_sbml_model(str("Model_Pvulgaris_CYC.sbml"))

### Print list of metabolites-no-formula

In [ ]:
count=0
metsNotFormula=[]
for met in modelPvul.metabolites:
    if met.formula == "":
        count+=1
        print(met.id)
        metsNotFormula.append(met.id)
        #print(modelPvul.metabolites.get_by_id(met.id).reactions)
print(count)

### Chemical formula filling

In [ ]:
### Example of chemical formula filling
modelPvul.metabolites.get_by_id("_14-alpha-methylsteroids_c").formula = "C20H33X"
modelPvul.metabolites.get_by_id("Red-NADPH-Hemoprotein-Reductases_c").formula = "C12H11N4O2X"
modelPvul.metabolites.get_by_id("_14a-Hydroxymethylsteroids_c").formula = "C20H33OX"
modelPvul.metabolites.get_by_id("Ox-NADPH-Hemoprotein-Reductases_c").formula = "C12H9N4O2X"

### Check number of atoms of compounds in reactions and mass balance

In [ ]:
for metabolite, coefficient in r._metabolites.items():
    print(str(metabolite) + ": " + str(metabolite.elements))
    
r.check_mass_balance()

### Chemical formula filling with a dictionary from MetaCyc service 


In [ ]:
# You must have a dataframe where shows the Metacyc ID and Kegg ID, along with chemical formula taken from another GEM obtained with kegg. With this table you
# can proceed to substitute the chemical formula of the metabolites in the model with the ones in the table.
translate=pd.read_csv('meta_kegg_formula')
translate.drop(columns=translate.columns[0])



In [ ]:
notrecog=[]
for i in range(len(translate)):
    for j in range(len(metsNotFormula)):
        c=(translate.iloc[i]['Metacyc']+'_c')
        if c == metsNotFormula[j]:
            print(c)
            formula=translate.iloc[i]['Formula']
            formula=str(formula)
            #print(c + '  '  + formula)
            last=formula[-1]
            sublast=formula[-2]
            if last=='R':
                forlen=len(formula)
                formula = formula.replace(formula[forlen-1:],'X')
                modelPvul.metabolites.get_by_id(c).formula = formula
                #print(c + '  '  + formula)
            elif sublast=='R':
                forlen=len(formula)
                formula = formula.replace(formula[forlen-2:],'X')
                modelPvul.metabolites.get_by_id(c).formula = formula
                #print(c + '  '  + formula)
            else:
                modelPvul.metabolites.get_by_id(c).formula = formula

### Retrieve unbalanced reactions

In [ ]:
count=0
for i in modelPvul.reactions:
    if i.check_mass_balance() !={}:
        print(i.id)
        count+=1
print(count)

### Check mass balance of reactions associated to a certain compound

In [ ]:

conteobln=0
conteo=0
for i in modelPvul.metabolites.get_by_id("ACP_c").reactions:
    conteo+=1
    if i.check_mass_balance() !={}:
        print('==============UNBALANCED RXNS============' + i.id + '\n')
        conteobln+=1
        mb=i.check_mass_balance()
        for metabolite, coefficient in i._metabolites.items():
            print(str(metabolite) + ": " + str(metabolite.elements))
        print("___________________MASS BALANCE   "+str(mb))

print('DESBALANCEADO = ' +str(conteobln) )            
print('BALANCEADO = '+ str(conteo-conteobln))